In [ ]:
import chainer
from chainer.dataset import convert

import melanoma

In [ ]:
device = 0
batch_size = 32
width = 224
height = 224
epoch = 10
output_dir = "results"
transfered_weights = None

model = melanoma.models.EfficientNet()
loss_func = model.loss_func
optimizer = chainer.optimizers.Adam()

if transfered_weights:
    chainer.serializers.load_npz(transfered_weights)

if device >= 0:
    model.to_gpu(device)

optimizer.setup(model)

train_ds, val_ds, test_ds = melanoma.utility.start_pdb(
    melanoma.dataset.DatasetBuilder(img_size=(width, height), n_classes=2).build
)

train_itr = chainer.iterators.MultiprocessIterator(train_ds, batch_size, repeat=True)
val_itr = chainer.iterators.MultiprocessIterator(val_ds, batch_size, repeat=False)
test_itr = chainer.iterators.MultiprocessIterator(test_ds, batch_size, repeat=False)

updater = chainer.training.StandardUpdater(
    train_itr, optimizer, converter=convert.concat_examples, loss_func=loss_func, device=device
)
evaluator = chainer.training.extensions.Evaluator(
    val_itr, model, device=device, eval_func=loss_func
)
trainer = melanoma.trainer.TrainerBuilder(updater, epoch ,evaluator, output_dir).build()

In [ ]:
melanoma.utility.start_pdb(
    trainer.run
)